In [1]:
## update the four bmp image folders, where:
## because only delay phase have bad medulla and cortex seperation, in general, the label of kidney is not used except annotation
'''
json_dict['labels'] = {
    "background": "0",
    "Tumor": "1",
    "Medulla": "2",
    "Cortex": "3",
    "Kidney": "4",
}
'''

'\njson_dict[\'labels\'] = {\n    "background": "0",\n    "Tumor": "1",\n    "Medulla": "2",\n    "Cortex": "3",\n    "Kidney": "4",\n}\n'

In [2]:
'''
# List of subdirectories to add
subdirectories = ["Kidney", "Tumor", "Cortex", "Medulla"]

# Iterate through each case_xxxxx directory
for dir in os.listdir("KiTS-bmp"):
    if dir.startswith("case_"):
        # Create directory A
        dir_A = os.path.join("FYP-KiTS/KiTS-bmp", dir, "A")
        os.makedirs(dir_A, exist_ok=True)

        # Create subdirectories in directory A
        for subdirectory in subdirectories:
            os.makedirs(os.path.join(dir_A, subdirectory), exist_ok=True)

        # Create directory D
        dir_D = os.path.join("FYP-KiTS/KiTS-bmp", dir, "D")
        os.makedirs(dir_D, exist_ok=True)

        # Create subdirectories in directory D
        for subdirectory in subdirectories:
            os.makedirs(os.path.join(dir_D, subdirectory), exist_ok=True)
'''

'\n# List of subdirectories to add\nsubdirectories = ["Kidney", "Tumor", "Cortex", "Medulla"]\n\n# Iterate through each case_xxxxx directory\nfor dir in os.listdir("KiTS-bmp"):\n    if dir.startswith("case_"):\n        # Create directory A\n        dir_A = os.path.join("FYP-KiTS/KiTS-bmp", dir, "A")\n        os.makedirs(dir_A, exist_ok=True)\n\n        # Create subdirectories in directory A\n        for subdirectory in subdirectories:\n            os.makedirs(os.path.join(dir_A, subdirectory), exist_ok=True)\n\n        # Create directory D\n        dir_D = os.path.join("FYP-KiTS/KiTS-bmp", dir, "D")\n        os.makedirs(dir_D, exist_ok=True)\n\n        # Create subdirectories in directory D\n        for subdirectory in subdirectories:\n            os.makedirs(os.path.join(dir_D, subdirectory), exist_ok=True)\n'

In [1]:
import cv2
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# test example usage:
# Define input and output directories
base_dir = Path('/home/rusk/projects/FYP-KiTS/bmpin')
output_dir = Path('/home/rusk/projects/FYP-KiTS/KiTS')

In [3]:
def extract_color(image, color_name):
    # Convert image to HSL color space
    hsl_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    
    # Define color ranges for yellow, green, and blue in HSL space
    color_ranges = {
        # define the color not include the red for left button text in bmp images (not 0 in Humidity)
        'not_red': ((1, 100, 100), (180, 255, 255))
    }
    
    # Get lower and upper bounds for the specified color
    lower_bound, upper_bound = color_ranges[color_name]
    
    # Create a mask using the specified color range
    mask = cv2.inRange(hsl_image, lower_bound, upper_bound)
    
    return mask

In [4]:
def bmp2nifti(image_dir, output_dir, name, save=True):
    # Get all BMP files in the specified directory
    bmp_files = [file for file in os.listdir(image_dir) if file.endswith('.bmp')]
    bmp_files.sort()  # Sort files to ensure consistent order
    
    # Load the first BMP image to get dimensions
    first_image = cv2.imread(os.path.join(image_dir, bmp_files[0]))
    height, width = first_image.shape[:2]
    
    # Initialize an empty array to store masks
    masks = np.zeros((len(bmp_files), height, width), dtype=np.uint8)
    
    # Iterate through all BMP files and extract masks
    for idx, bmp_file in enumerate(bmp_files):
        image = cv2.imread(os.path.join(image_dir, bmp_file))
        not_red_mask = extract_color(image, 'not_red')
        
        # Combine masks into a single mask
        combined_mask = np.zeros_like(not_red_mask)

        # This order keep the order to show tumor in the first layer
        combined_mask[not_red_mask > 0] = 1  # mask label except the background
        
        # Store the combined mask
        masks[idx] = combined_mask
    
    # Create a 3D volume from the stacked masks
    volume = np.transpose(masks, (2, 1, 0))

    # Rotate the volume anticlockwise by 90 degrees
    # volume = np.rot90(volume, k=1, axes=(0, 1))
    
    # Save the volume as a NIfTI file
    nii_save_path = os.path.join(output_dir, name + '.nii.gz')
    if save:
        # Save combined mask as .nii.gz file
        affine = np.eye(4)  # Assuming identity affine matrix
        # header = nib.Nifti1Header()
        # header.set_data_shape(combined_mask.shape)
        # nifti_img = nib.Nifti1Image(volume, affine, header)
        nifti_img = nib.Nifti1Image(volume, affine)
        
        nib.save(nifti_img, output_dir / (name + '.nii.gz'))
        print(f"Segmentation saved as {nii_save_path}")
    else:
        print(f"Segmentation FAILED TO save as {nii_save_path}")
    return volume

In [5]:
def convert_folders(input_dir, output_dir):
    # Define structures
    structures = ['Kidney', 'Tumor', 'Cortex', 'Medulla']
    
    # Process each structure
    for structure in structures:    
        # Perform segmentation
        segmentation_volume = bmp2nifti(input_dir / structure, output_dir, structure.lower())
    
    
    # convert seperated masks to one multi-object segmentation.nii.gz file
    # Load the segmentation files as arrays
    kidney_segmentation = nib.load(output_dir / 'kidney.nii.gz').get_fdata().astype(np.uint8)
    tumor_segmentation = nib.load(output_dir / 'tumor.nii.gz').get_fdata().astype(np.uint8)
    cortex_segmentation = nib.load(output_dir / 'cortex.nii.gz').get_fdata().astype(np.uint8)
    medulla_segmentation = nib.load(output_dir / 'medulla.nii.gz').get_fdata().astype(np.uint8)
    
    # Create a combined segmentation
    combined_data = np.zeros_like(kidney_segmentation)
    
    # Assign labels to different masks
    # combined_data[kidney_segmentation != 0] = 4
    combined_data[cortex_segmentation != 0] = 3
    combined_data[medulla_segmentation != 0] = 2
    combined_data[tumor_segmentation != 0] = 1
    
    # Save the combined segmentation
    combined_segmentation = nib.Nifti1Image(combined_data, nib.load(output_dir / 'kidney.nii.gz').affine)
    # nib.save(combined_segmentation, name + phase + 'segmentation.nii.gz')
    nib.save(combined_segmentation, output_dir / 'segmentation.nii.gz')
    print("success to save as segmentation.nii.gz")

    # delete unnessesary files
    os.remove(output_dir / 'kidney.nii.gz')
    os.remove(output_dir / 'tumor.nii.gz')
    os.remove(output_dir / 'cortex.nii.gz')
    os.remove(output_dir / 'medulla.nii.gz')

In [7]:
# Iterate through each case folder in KiTS-dicom
for case_folder in base_dir.glob('case_*'):
    case_name = case_folder.name
    # Process subfolders A and D
    for subfolder in ['A', 'D']:
        label_in_path = case_folder / subfolder
        out_path = Path(output_dir / case_name / subfolder)
        # Check if the subfolder exists
        if label_in_path.exists():
            if os.listdir(label_in_path / 'Tumor'):
                # Convert BMP to NIfTI
                convert_folders(label_in_path, out_path)
                
            else:
                print("Canot find the BMP files in " + str(label_in_path))

Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/A/kidney.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/A/tumor.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/A/cortex.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/A/medulla.nii.gz
success to save as segmentation.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/D/kidney.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/D/tumor.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/D/cortex.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00015/D/medulla.nii.gz
success to save as segmentation.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00014/A/kidney.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00014/A/tumor.nii.gz
Segmentation saved as /home/rusk/projects/FYP-KiTS/KiTS/case_00014/A/cortex.nii.gz
Segmentati